# Estudio de redes neuronales profundas para la clasificación y caracterización de prendas de vestir

## Víctor Herrera Delgado

El código dispuesto a continuación consiste en una versión limpia de los pasos que realicé para el desarrollo de mi trabajo de fin de título para la clasificación y caracterización de prendas de ropa y elementos adicionales del mundo de la moda. 

Este proyecto te puede servir como introducción a la hora de crear tu propia red neuronal para clasificar.

En concreto en este documento tratamos el uso de redes neuronales convolucionales que siguen la estructura VGG, usando como herramienta Pytorch.

### Uso del código

El código se divide en dos partes:

- Inicialización de las clases, funciones y variables que se utilizarán. En cada bloque se explicará en que consiste cada uno y su funcionamiento.
- Ejecución del entrenamiento y de otras fases para el análisis de los resultados.
- Optimización de hiperparámetros.
- Lectura avanzada de resultados.


--------------------------

# Inicialización del proyecto 
#### (Ejecutar todos los bloques hasta la siguiente mitad para el uso directo del código)

## Paso 1: Importación de librería e inicialización de variables globales


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import skopt
from skopt import callbacks
from skopt import dump, load
from skopt.callbacks import CheckpointSaver
import math
import PIL
import pandas as pd
import os
import time
import copy
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import models
from torch.optim import lr_scheduler
from PIL import Image
from enum import Enum
from livelossplot import PlotLosses
import sys
import seaborn as sn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

class Mode(Enum):
    GENERAL = 0  #Modelo modificable por el usuario
    VGG = 1      #Uso único de la clasificación de VGG
    MIDVGG3 = 2  #Uso único de fine-tunning de 3 capas de VGG
    BLANKVGG = 3 #Red con estructura VGG normal
    MIDVGG2 = 4  #Uso único de fine-tunning de 2 capas de VGG
    MIDVGG1 = 5  #Uso único de fine-tunning de 1 capa de VGG

MODE = Mode.VGG

## Transformación de imágenes

Si los datos se toman directamente desde el dataset, la variable "transform" permitirá adaptar los datos normalizados para la entrada a las capas tipo VGG.
Si se usa la variable "dataAugmentationTransform", los datos se rotaran, daran la vuelta y recortaran aleatoriamente para aumentar la cantidad de datos que componen el dataset durante el entrenamiento.

In [ ]:
DATA_PATH = "/storage/datasets/fashion-dataset/" 
img_width, img_height, _ = 32, 32, 3
image_size = (img_width, img_height)
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([
        transforms.Resize([224,224]),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

dataAugmentationTransform = transforms.Compose([
    transforms.Resize(235),
    #transforms.CenterCrop(224),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20, resample=False, expand=False, center=None),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])



## Dataset
El dataset se guarda en un objeto del tipo dataset y cuenta con el siguiente grupo de variables:
- *path*: El path del dataset original.
- *file_list*: la lista de imágenes que se pueden leer.
- *division*: la categoría por la que se dividen.
- *transform*: el transformador que actua sobre sus imágenes.
- *chargedInVgg*: boolean para indicar si salen de el las imágenes o los resultados de pasar las imágenes por la parte convolutiva.
- *PreResults*: resultados de pasar las imágenes por la parte convolutiva.
El dataset se inicia pasandole un objeto transform y el campo por el que se clasificarán. Al cargarlo, se comprobará la fiabilidad de las imágenes.  
Para obtener los resultados de pasarla por la parte convolutiva de la VGG (o por una parte de la red) hay que ejecutar la función preNet de la clase, que pasándole la parte convolutiva de la red se encargará de ello. Podemos con la función changeSource(bool) especificar que valores queremos obtener del dataset, si las imágenes (bool = False) o los resultados de pasar por la parte de la red con anterioridad (bool = True).

#### (Recuerde hacer en la función "preparationForPreConvolutive" el cambio correspondiente de ficheros para adaptarla a usted)

In [ ]:
class TheDataset(Dataset):
    def __init__(self,transform=None,division=None):
        self.path= DATA_PATH
        self.file_list = pd.read_csv(self.path + "styles.csv", error_bad_lines=False)
        self.file_list['image'] =self.file_list.apply(lambda row: self.path + "images/"+ str(row['id']) + ".jpg", axis=1)
        self.file_list = self.file_list.reset_index(drop=True)
        self.division = division
        self.classes= tuple(self.file_list[division].unique())
        self.transform = transform
        self.tryDataset()
        self.chargedInVGG = 0
        self.PreResults = []

    def __len__(self):
        return len(self.file_list)
        
    def __getitem__(self,idx):
        if self.chargedInVGG==Mode.GENERAL or self.chargedInVGG==Mode.BLANKVGG:
            row = self.file_list[idx]
            imageClass = self.classes.index(row[self.division])
            image = Image.open(row['image'])
            if self.transform:
                    image= self.transform(image)
            return image, imageClass 
        else:
            row = self.file_list[idx]
            imageClass = self.classes.index(row[self.division])
            if self.chargedInVGG == Mode.MIDVGG3:
                return  torch.load(row['midConvolutive3']), imageClass
            elif self.chargedInVGG == Mode.MIDVGG2:
                return  torch.load(row['midConvolutive2']), imageClass
            elif self.chargedInVGG == Mode.MIDVGG1:
                return  torch.load(row['midConvolutive1']), imageClass
            elif self.chargedInVGG == Mode.VGG:
                return torch.load(row['completeConvolutive']), imageClass
    
    def tryDataset(self):
        cont = 0
        files=[]
        for index, row in self.file_list.iterrows():
            if not os.path.isfile(row['image']) : continue
            if row['masterCategory'] == "Free Items" or row['subCategory'] == "Free Gifts" or row['articleType'] == "Free Gifts" or (row['baseColour'] == "nan") or str(row['season']) == "nan" or str(row['usage']) == "nan":
                continue
            files.append(row)
            cont += 1
    

        self.file_list = files
    def changeSource(self,mode):
        self.chargedInVGG = mode
    
    def preparationForPreConvolutive(self,mode):
        if mode == Mode.MIDVGG3:
            for index in range(len(self.file_list)):
                self.file_list[index]['midConvolutive3'] = "/storage/midConvolutiveResults3_2/file" + str(self.file_list[index]['id']);
        elif mode == Mode.MIDVGG2:
            for index in range(len(self.file_list)):
                self.file_list[index]['midConvolutive2'] = "/storage/midConvolutiveResults2_2/file" + str(self.file_list[index]['id']);
        elif mode == Mode.MIDVGG1:
            for index in range(len(self.file_list)):
                self.file_list[index]['midConvolutive1'] = "/storage/midConvolutiveResults1_2/file" + str(self.file_list[index]['id']);
        elif mode == Mode.VGG:
            for index in range(len(self.file_list)):
                self.file_list[index]['completeConvolutive'] = "/storage/convolutiveResultsNoFree_2/file" + str(self.file_list[index]['id']);
   
    def completePreNet(self, notClassNet,folderPath):
        #DATA_PATH = "/storage/datasets/fashion-dataset/"
        files = []
        count = 0
        
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        print(device)
        notClassNet.to(device)
        notClassNet.cuda()
        with torch.no_grad():
            for index in range(len(self.file_list)):
                if index % 5000 == 0: 
                    print(index)
                imgAndClass = []
                theImg,theClass = self.__getitem__(index)
                if(theClass == 4):
                    print(self.file_list[index])
                theImg = torch.tensor([theImg.tolist()])
                theImg = theImg.to(device) 
                output = notClassNet(theImg)
                output = torch.flatten(output, 1)
                output = torch.tensor(output.tolist()[0])
                torch.save(output,folderPath + "file" + str(self.file_list[index]['id']) );
                self.file_list[index]['completeConvolutive'] = folderPath +"file" + str(self.file_list[index]['id']);
                torch.cuda.empty_cache()
        torch.cuda.empty_cache()
            
    def midPreNet(self, notClassNet,folderPath):
        count = 0
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        print(device)
        notClassNet.to(device)
        notClassNet.cuda()
        with torch.no_grad():
            for index in range(len(self.file_list)):
                if index % 5000 == 0: 
                    print(index)
                
                imgAndClass = []
                theImg,theClass = self.__getitem__(index)
                theImg = torch.tensor([theImg.tolist()])
                theImg = theImg.to(device) 
                output = notClassNet(theImg)
                output = torch.tensor(output.tolist()[0])
                torch.save(output,folderPath + "file" + str(self.file_list[index]['id']) );
                self.file_list[index]['midConvolutive'] = folderPath + "file" + str(self.file_list[index]['id']);
                torch.cuda.empty_cache()
        torch.cuda.empty_cache()
        

## Dataloader
#### Ejemplo de creación
El dataloader será el que proporcionará los distintos elementos del dataset al modelo de la red, además de dividirlos en entrenamiento y validación y configurar los parámetros de tamaño de batch y la aleatoriedad de los elementos del dataset.

In [ ]:
#DATALOADER
def createDataloader(batch_size = 4, num_workers = 0, trainSize = 30000):
    shuffle = False
    valSize = len(theDatasetPr)-trainSize
    testLoader = DataLoader(dataset=test_ds,
                            shuffle=shuffle,
                            batch_size=batch_size,
                            num_workers=num_workers)
    validLoader = DataLoader(dataset=valid_ds,
                            shuffle=shuffle,
                            batch_size=batch_size,
                            num_workers=num_workers)
    print(validLoader.batch_size)
    dataloaders = {'train': testLoader, 'val': validLoader}
    dataset_sizes = {'train': trainSize, 'val': valSize}
    return dataloaders,dataset_sizes
#def resetNetModel():
    

## Error de entrenamiento
### Ajuste del error de entrenamiento a tener en cuenta
En función del modelo que se le pase, se obtendrán los parámetros de la obtención del error en función de los cuales se corregirá el modelo para que cumpla el objetivo propuesto. A la función se le da como parámetros el modelo a entrenar y el learning rate deseado (0.001 por defecto). Devuelve las variables Criterion, el optimizador y el scheduler.

Se utiliza para la corrección el descenso por el gradiente estocástico


In [ ]:
def createTrainingError(model,lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer= optim.SGD(model.parameters(), lr, momentum=0.9)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
    return criterion,optimizer,exp_lr_scheduler

## Modelos
Dependiendo de la elección realizada (tipo de red a utilizar) se utilizará una configuración u otra. Las configuraciones son modificables pero en general son:
- General: Se tiene una red simple "GeneralModel" que puede ser modificable por el usuario.
- BLANKVGG: Devuelve una estructura VGG para su entrenamiento.
- VGG: Una red basada en una VGG preentrenada y a la cual se le puede modificar su parte clasificadora. Se puede modificar en "VGGModel". 
- MidVGG1, MidVGG2, MidVGG3: Una red basada en la red obtenida al crear un modelo en VGG con la diferencia de que parte de sus capas convolutivas se entrenan, véase que se realiza fine-tunning. Se puede modificar la parte convolutiva seleccionada en "MidVGGModel". El número que sigue a MIDVGG depende del número de capas convolucionales que se modifiquen.

##### (Por defecto VGG y las MIDVGG están preparadas para entradas de datos ya preprocesadas por el dataset para disminuir su coste. Si desea entrenarla de forma convencional descomente los comentarios en sus respectivas funciones forward)


In [ ]:
def getClassifier(intro,outro,netArray,d_dropout=0.5):
    if(len(netArray) < 1): 
        return []
    newClassifier = [nn.Linear(intro, netArray[0]),nn.Dropout(p=d_dropout, inplace=False),nn.ReLU(inplace=True)]
    for i in range(len(netArray)):
        if(i == 0):
            continue
        else:  
            newClassifier = newClassifier + [nn.Linear(netArray[i-1], netArray[i]),nn.Dropout(p=d_dropout, inplace=False),nn.ReLU(inplace=True)]
    newClassifier = newClassifier + [nn.Linear(netArray[len(netArray) - 1], outro)]
    return newClassifier        
    
class GeneralModel (nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(3,6,3)
        #30x30x6
        self.pool1 = nn.MaxPool2d(2,2)
        #15x15x6
        self.conv2 = nn.Conv2d(6,16,2)
        #14x14x16
        self.conv3 = nn.Conv2d(16,32,5)
        #10x10x16
        self.fc1 = nn.Linear(32*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,len(theDatasetPr.classes))

    def forward(self,x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = F.relu(self.conv2(x))
        x = self.pool1(F.relu(self.conv3(x)))
        x = x.view(-1,32*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class VGGModel (nn.Module):
    def __init__(self,net,dropout,netArray):
        super(VGGModel,self).__init__()
        self.features = net.features
        for param in self.features.parameters():
            param.require_grad = False
        nFeats = net.classifier[6].in_features
        feats = list(net.classifier.children())[:-7]
        nFeats = net.classifier[0].in_features
        feats.extend(getClassifier(nFeats,len(theDatasetPr.classes),netArray,dropout))
        self.avgpool = net.avgpool
        self.classifier = nn.Sequential(*feats)
    def forward(self,x):
        #x = self.features(x)
        #x = self.avgpool(x)
        x = self.classifier(x)
        return x    

class MidVGG3Model (nn.Module):
    def __init__(self,net):
        super(MidVGG3Model,self).__init__()
        self.features = net.features[:-10]
        self.newFeatures = net.features[-10:]
        for param in self.newFeatures.parameters():
            param.require_grad = True
        self.avgpool = net.avgpool
        self.classifier = net.classifier
    def forward(self,x):
        #x = self.features(x)
        x = self.newFeatures(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
    
class MidVGG2Model (nn.Module):
    def __init__(self,net):
        super(MidVGG2Model,self).__init__()
        self.features = net.features[:-7]
        self.newFeatures = net.features[-7:]
        for param in self.newFeatures.parameters():
            param.require_grad = True
        self.avgpool = net.avgpool
        self.classifier = net.classifier
    def forward(self,x):
        #x = self.features(x)
        x = self.newFeatures(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
    
class MidVGG1Model (nn.Module):
    def __init__(self,net):
        super(MidVGG1Model,self).__init__()
        self.features = net.features[:-4]
        self.newFeatures = net.features[-4:]
        for param in self.newFeatures.parameters():
            param.require_grad = True
        self.avgpool = net.avgpool
        self.classifier = net.classifier
    def forward(self,x):
        #x = self.features(x)
        x = self.newFeatures(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
    
def generateModel(MODE,dropout,netArray):
    newModel = None
    print(MODE)
    if(MODE == Mode.GENERAL):
        newModel = GeneralModel()
        return newModel
    if(MODE == Mode.VGG):
        newModel = models.vgg16_bn(pretrained=True)
        newModel = VGGModel(newModel,dropout,netArray)
        return newModel
    if(MODE == Mode.BLANKVGG):
        newModel = models.vgg16_bn(pretrained=True)
        feats = list(newModel.classifier.children())[:-1]
        feats.extend([nn.Linear(4096, len(theDatasetPr.classes))])
        newModel.classifier = nn.Sequential(*feats)
        return newModel
    if(MODE == Mode.MIDVGG3):
        newModel = models.vgg16_bn(pretrained=True)
        MODE = Mode.VGG
        newModel = MidVGG3Model(generateModel(MODE,dropout,netArray))
        MODE = Mode.MIDVGG3
        return newModel
    if(MODE == Mode.MIDVGG2):
        newModel = models.vgg16_bn(pretrained=True)
        MODE = Mode.VGG
        newModel = MidVGG2Model(generateModel(MODE,dropout,netArray))
        MODE = Mode.MIDVGG2
        return newModel
    if(MODE == Mode.MIDVGG1):
        newModel = models.vgg16_bn(pretrained=True)
        MODE = Mode.VGG
        newModel = MidVGG1Model(generateModel(MODE,dropout,netArray))
        MODE = Mode.MIDVGG1
        return newModel
    return newModel
    
def createModel(MODE=Mode.GENERAL,dropout=0.5,netArray=[4096,4096]):
    newModel = generateModel(MODE,dropout,netArray)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)
    newModel.to(device)
    newModel.cuda()
    return newModel

### Entrenamiento de la red
Pasándole los parámetros necesarios realizamos un entrenamiento en el cual para cada época se entrena y se valida la red, mostrando los resultados de pérdida y precisión para cada caso. Se ha incorporado la librería liveloss para poder mostrar estos resultados como gráficas de los resultados a tiempo real. 
La función guardará la red que mejores resultados haya obtenido y no cambiará hasta encontrar una que mejore su ejecución en la validación.
Al finalizar, devuelve el modelo más óptimo obtenido, así como la media de las perdidas obtenidas, que tiene su uso a la hora de buscar los hiperparámetros óptimos.



In [ ]:
#Función adicional para imprimir el resultado en el fichero especificado aquí
def printInFile(time_elapsed, best_acc, the_real_acc, the_loss,the_train_acc,the_train_loss):
    print("ENTRAMOS")
    with open('LASTRESULTS.txt', 'w') as f:
        print("TRAINING")
        print('Training complete in {:.0f}m {:.0f}s'.format(
            time_elapsed // 60, time_elapsed % 60),file=f)
        print('Best val Acc: {:4f}'.format(best_acc),file=f)
        
        print("ACCURACY")
        print("VALIDATION",file=f)
        print("ACCURACY HISTORY",file=f)
        for i in the_real_acc: 
            print(i.item(),file=f) 
        print("\n\nLOSS HISTORY",file=f)
        for x in the_loss: 
            print(x,file=f) 
            
        print("\n\nTRAINING",file=f)
        print("ACCURACY HISTORY",file=f)
        for i in the_train_acc: 
            print(i.item(),file=f) 
        print("\nLOSS HISTORY",file=f)
        for x in the_train_loss: 
            print(x,file=f) 
        print("FIN")
    

In [ ]:
#Basado en la documentación de pytorch
def train_model(model, thecriterion, theoptimizer, thescheduler,dataloaders=None,dataset_sizes=None, num_epochs=25):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    since = time.time()
    liveloss = PlotLosses()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_loss = 1000000.0
    the_loss = []
    the_acc = []
    the_real_acc = []
    the_train_acc = []
    the_train_loss = []
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        logs={}
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            batchCont = 0
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                theoptimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = thecriterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        theoptimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                batchCont+=1
                if batchCont % 250 == 0: 
                    print("batch = ",batchCont)
            if phase == 'train':
                thescheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            prefix = ''
            if phase == 'val':
                prefix = 'val_'

            logs[prefix + 'log loss'] = epoch_loss
            logs[prefix + 'accuracy'] = epoch_acc

            # deep copy the model
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), "./mejorActual")
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                #best_model_wts = copy.deepcopy(model.state_dict())
                #torch.save(model.state_dict(), "./mejorActual")
            if phase == 'val':
                the_acc.append((1.0 - epoch_acc).double())
                the_real_acc.append(epoch_acc)
                the_loss.append(epoch_loss)
                liveloss.update(logs)
                liveloss.send()
            if phase == 'train':
                the_train_acc.append(epoch_acc)
                the_train_loss.append(epoch_loss)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    printInFile(time_elapsed, best_acc, the_real_acc, the_loss, the_train_acc,the_train_loss)
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_acc.item()

## Estudios individuales
A continuación se exponen las funciones dedicadas a observar la sensibilidad de un modelo para una red entrenada, obteniendo sus resultados de precisión para cada categoría en el conjunto de categorías actual en el dataset.

In [ ]:
def class_success_val(model,dataloaders):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #netPr.to(device)
    lengu = len(theDatasetPr.classes)
    class_correct = list(0. for i in range(lengu))
    class_total = list(0. for i in range (lengu))
    with torch.no_grad():
        for data in dataloaders['val']:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs,1);
            c = (predicted == labels).squeeze()
            for i in range(len(labels)):
                #if(len(labels) < 4): continue
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] +=1

    #for i in range(lengu):
    #  print('Accuracy of %5s : %2d %%' % (
    #      theDatasetPr.classes[i],100 * class_correct[i]/(max(class_total[i],1))), "Total = ", class_total[i])
    return class_correct,class_total

def class_success_train(model,dataloaders):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #netPr.to(device)
    lengu = len(theDatasetPr.classes)
    class_correct = list(0. for i in range(lengu))
    class_total = list(0. for i in range (lengu))
    with torch.no_grad():
        for data in dataloaders['train']:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs,1);
            c = (predicted == labels).squeeze()
            for i in range(len(labels)):
                #if(len(labels) < 4): continue
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] +=1

    #for i in range(lengu):
    #  print('Accuracy of %5s : %2d %%' % (
    #      theDatasetPr.classes[i],100 * class_correct[i]/(max(class_total[i],1))), "Total = ", class_total[i])
    return class_correct, class_total

def class_success_total(dataloaders):
    lengu = len(theDatasetPr.classes)
    class_total = list(0. for i in range (lengu))
    with torch.no_grad():
        for data in dataloaders['train']:
            _, labels = data
            for i in range(len(labels)):
                label = labels[i]
                class_total[label] +=1
        for data in dataloaders['val']:
            _, labels = data
            for i in range(len(labels)):
                label = labels[i]
                class_total[label] +=1
    #for i in range(lengu):
    #    print(theDatasetPr.classes[i],"    Total = ", class_total[i])
    return class_total

def class_success(model,dataloaders):
    correctVal, totalVal = class_success_val(model,dataloaders)#createDataloader(batch_size = 64, num_workers = 0, trainSize = 30000))
    print(" ")
    correctTrain, totalTrain = class_success_train(model,dataloaders) #createDataloader(batch_size = 64, num_workers = 0, trainSize = 30000))
    print(" ")
    totalClass = class_success_total(dataloaders)#createDataloader(batch_size = 64, num_workers = 0, trainSize = 30000))
    #print("Clase\t\t\tTotal\t\t\tPrecision Val\t\t\tPrecision Train\t\t\tTotal Val\t\t\tTotal Train")
    for i in range(len(theDatasetPr.classes)):
        print(
          theDatasetPr.classes[i], "\t\t\t",totalClass[i] ,
            "\t\t\n Val%",
            (100 * correctVal[i]/(max(totalVal[i],1))),
            "\t\t   Train%",
            (100 * correctTrain[i]/(max(totalTrain[i],1))),
            "\t\t   VT",
            totalVal[i],
            "\t\t   TT",
            totalTrain[i], 
            "\n"
            )
    return totalClass,correctVal,totalVal,correctTrain,totalTrain
def confusion_prepare(model,dataloaders):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #netPr.to(device)
    lengu = len(dataloaders['val'])
    print("lengu= ", lengu)
    well_predicted = []#list(0. for i in range(lengu))
    prediction = []#list(0. for i in range (lengu))
    model.eval()
    with torch.no_grad():
        for data in dataloaders['val']:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs,1);
            for i in range(len(labels)):
                prediction.append(theDatasetPr.classes[predicted[i].item()])
                well_predicted.append(theDatasetPr.classes[labels[i].item()])
            #    label = labels[i]
            #    class_correct[label] += c[i].item()
            #    class_total[label] +=1
    return well_predicted,prediction

--------------
# Sección de entrenamiento

Establecemos el tipo de red que queremos entrenar

In [ ]:
MODE = Mode.MIDVGG3

### Dataset
Preparamos el dataset pasandole el transformador adecuado y la categoría por la cual vamos a clasificar. En caso de necesitar cambiar categoría debemos volver a ejecutar este bloque

In [ ]:
theDatasetPr = TheDataset(transform=transform,division="masterCategory")
theDatasetPr.changeSource(MODE)

Ejecuta el bloque a continuación solo si quieres entrenar con datos preprocesados

In [ ]:
theDatasetPr.changeSource(Mode.GENERAL)
theDatasetPr.preparationForPreConvolutive(MODE)
theDatasetPr.changeSource(MODE)

## Dataloaders

Preparamos los dataloader para suministrar la información. En caso de querer cambiar el tamaño de batch o la distribución de elementos cambia los hiperparámetros.

In [ ]:
trainSize= 30000
shuffle = True
batch_size = 32
num_workers = 0
valSize = len(theDatasetPr)-trainSize
dataloader = DataLoader(dataset=theDatasetPr,
                            shuffle=shuffle,
                            batch_size=batch_size,
                            num_workers=num_workers)

#### (manual_seed permitirá que entrenamiento y validación se dividan siempre de la misma manera)

In [ ]:
torch.manual_seed(0)

test_ds, valid_ds = torch.utils.data.random_split(dataloader.dataset, (trainSize, valSize))

dataloads, dataset_sis = createDataloader(batch_size = batch_size, num_workers = 0, trainSize= trainSize)#trainSize = 30000)

## Modelo y error

Creación del modelo y ajuste de la corrección del error.

A createModel además de pasarle el modo se le puede pasar el dropout (por defecto 0.5) aplicar en la clasificación. También se le puede pasar una tupla que contenga el valor de entrada y salida de las capas intermedias. Por ejeplo si queremos una capa de entrada 512 y salida 512 pondremos \[512, 512\]. En caso de querer añadir una segunda capa añada su número de salidas con otro elemento de la tupla. Por defecto tiene una capa de 4096x4096.

A los parámetros del error, además del modelo creado debemos pasarle el learning rate.

In [ ]:
classNet = createModel(MODE,0.5)
crit,opts, exp_lr_scheds = createTrainingError(classNet,lr=0.003920274771701523)
print(classNet)

#### (En caso de querer cargar los pesos de una red anterior, puede usar la siguiente función)

In [ ]:
classNet.load_state_dict(torch.load("./modeloAntiguo"))

## Entrenamiento

Para ponerla a funcionar debe pasarle los parámetros creados durante las últimas funciones y el número de épocas.

Se le devolverá el modelo que mejor exactitud haya logrado y el valor de esa exactitud.
Los mejores pesos obtenidos serán introducidos en el sistema con el nombre declarado en la función de entrenamiento.
La evolución será impresa en el archivo especificado en la función de impresión. **Para el uso de la impresión debe borrar el archivo anterior** 

In [ ]:
classNet,bestAcc = train_model(classNet, crit, opts, exp_lr_scheds,dataloads,dataset_sis,num_epochs=10)

## Revisión de la sensibilidad

La función a continuación le suministrará datos acerca del número de elementos de cada conjunto y la sensibilidad que logró para cada posible clasificación de la categoría.


In [ ]:
retTotalClass,retCorrectVal,retTotalVal,retCorrectTrain,retTotalTrain = class_success(classNet,dataloads)

#### Añadido para comprobar si a sensibilidad de una clase está relacionada con el número de elementos que la componen

In [ ]:
valPercentage = []
trainPercentage = []
elementsTotalPercentage = []
for i in range(len(theDatasetPr.classes)):
    valPercentage.insert(i,100 * retCorrectVal[i]/max(retTotalVal[i],1))
    trainPercentage.insert(i,100 * retCorrectTrain[i]/max(retTotalTrain[i],1))
    elementsTotalPercentage.insert(i,100 * retTotalClass[i]/len(theDatasetPr))
xyz = np.array([retTotalClass,
                valPercentage,
                trainPercentage
                ])
xyz = np.corrcoef(xyz,rowvar= True);
print(xyz)

---------

# Optimización de hiperparámetros

### Inicialización común al entrenamiento

In [ ]:
theDatasetPr = TheDataset(transform=transform,division="masterCategory")
theDatasetPr.changeSource(MODE)

In [ ]:
theDatasetPr.changeSource(Mode.GENERAL)
theDatasetPr.preparationForPreConvolutive(MODE)
theDatasetPr.changeSource(MODE)

In [ ]:
trainSize= 30000
shuffle = True
batch_size = 32
num_workers = 0
valSize = len(theDatasetPr)-trainSize
dataloader = DataLoader(dataset=theDatasetPr,
                            shuffle=shuffle,
                            batch_size=batch_size,
                            num_workers=num_workers)
torch.manual_seed(0)
test_ds, valid_ds = torch.utils.data.random_split(dataloader.dataset, (trainSize, valSize))


### Definición de rangos

Aquí se especifica los rangos en los cuales van a oscilar nuestros hiperparámetros.

La variable global DaBestAcc se debe inicializar con el mínimo porcentaje no acertado de la red (1 - exactitud).

In [ ]:
SPACE = [
    skopt.space.Real(0.001, 0.05, name='learning_rate', prior='log-uniform') #0.001
    ,skopt.space.Categorical([16,32,64], name='batch_size' )
    ,skopt.space.Categorical([8192,4096,2048,1024,512],name='neurons1')
    ,skopt.space.Categorical([8192,4096,2048,1024,512],name='neurons2')
    #,skopt.space.Real(0.1, 0.9, name='dropout') 
    #,skopt.space.Categorical([8192,4096,2048,1024,512],name='optional_neurons')
]


global DaBestAcc
DaBestAcc = 0.06325128792215229

### Función objetivo

Función objetivo la cual queremos optimizar, véase el modelo de red. Guardará temporalmente el valor mínimo obtenido en global para hacer comparaciones y salvar en el sistema aquellos modelos que hayan hecho una mejor clasificación.

In [ ]:
def objective(params):
    torch.cuda.reset_max_memory_allocated()
    torch.cuda.reset_max_memory_cached()
    print(params)
    learning_rate  = params[0]
    the_batch_size = int(params[1])
    neurons1 = params[2]
    neurons2 = params[3]
    the_dropout =  0.5#params[4];
    optional_neurons= None
    #optional_neurons = params[5]

    dataloads, dataset_sis = createDataloader(batch_size = the_batch_size, num_workers = 0, trainSize= trainSize)#trainSize = 30000)
    classNet = None
    if(optional_neurons == None):
        classNet = createModel(MODE,the_dropout,[neurons1,neurons2])
    else:
        classNet = createModel(MODE,the_dropout,[neurons1,neurons2,neurons3])
    #classNet = nn.Sequential(*list(classNets.children())[2])
    print(classNet)

    
    
    crit,opts, exp_lr_scheds = createTrainingError(classNet,lr=learning_rate)
    
    classNet,loss = train_model(classNet, crit, opts, exp_lr_scheds,dataloads,dataset_sis,num_epochs=15)
    loss = 1.0 - loss
    print(loss)
    global DaBestAcc
    if (loss < DaBestAcc):
        DaBestAcc = loss
        print("NICE",DaBestAcc )
        torch.save(classNet.state_dict(), "./DaBestAcc")
    return loss



El siguiente bloque de código crea un checkpoint que sirve para tener constancia del progreso de la optimización

In [ ]:
checkpoint_saver = CheckpointSaver("./newCheckpoint1.pkl", compress=9)

#### (Si es la primera vez que optimizas, ejecuta este bloque)

In [ ]:
results = skopt.forest_minimize(objective, SPACE,n_calls=30,callback=[checkpoint_saver])
dump(results, 'newHp.pkl')

#### (Si no, puedes cargar un checkpoint a partir del cual seguir buscando los mejores valores)

In [ ]:
checkpoint_saver = CheckpointSaver("./newCheckpoint1.pkl", compress=9)
x0 = checkRes.x_iters
y0 = checkRes.func_vals

In [ ]:
results = skopt.forest_minimize(objective, SPACE,n_calls=30,x0=x0,y0=y0,callback=[checkpoint_saver])#,n_random_starts=3)
dump(results, 'newHp.pkl')

--------

# Estudio adicional

In [ ]:
theDatasetPr = TheDataset(transform=transform,division="masterCategory")
theDatasetPr.changeSource(MODE)

In [ ]:
theDatasetPr.changeSource(Mode.GENERAL)
theDatasetPr.preparationForPreConvolutive(MODE)
theDatasetPr.changeSource(MODE)

In [ ]:
trainSize= 30000
shuffle = True
batch_size = 32
num_workers = 0
valSize = len(theDatasetPr)-trainSize
dataloader = DataLoader(dataset=theDatasetPr,
                            shuffle=shuffle,
                            batch_size=batch_size,
                            num_workers=num_workers)
torch.manual_seed(0)
test_ds, valid_ds = torch.utils.data.random_split(dataloader.dataset, (trainSize, valSize))
dataloads, dataset_sis = createDataloader(batch_size = batch_size, num_workers = 0, trainSize= trainSize)

In [ ]:
classNet = createModel(MODE,0.5) 
classNet.load_state_dict(torch.load("./modeloAntiguo"))


### Matrix de confusión

In [ ]:
etiquetas, prediccion = confusion_prepare(classNet,dataloads)
print(len(etiquetas))
print(len(prediccion))

In [ ]:

labs = theDatasetPr.classes
print(labs)
print(type(labs[4]))
labs = tuple(x for x in labs if (x != 'Free Gifts' and type(x) != type(1.1)))
array = confusion_matrix(etiquetas, prediccion, labels=labs)
df_cm = pd.DataFrame(array, index = labs, columns = labs)
plt.figure(figsize = (20,15))
sn.set(font_scale=1)

heatmap = sn.heatmap(df_cm, annot=True,fmt='g')
snsplot = heatmap.get_figure()  


In [ ]:
# Guardado opcional de la gráfica
snsplot.savefig("heatMap.png")

### Exactitud

In [ ]:
accuracy_score(etiquetas, prediccion)

### Sensibilidad

In [ ]:
recall_score(etiquetas, prediccion, labels=theDatasetPr.classes, average=None)

### Precisión

In [ ]:
precision_score(etiquetas, prediccion, labels=theDatasetPr.classes, average=None)